In [1]:
import modin.pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
import keras.backend as K

import lime

Using TensorFlow backend.


In [2]:
training_dat = pd.read_pickle('./PROCESSED/training_dat2.pkl')
training_dat.columns

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

To request implementation, send an email to feature_requests@modin.org.


Index(['Loan Sequence Number', 'Loan Deliquency Within Year', 'Credit Score',
       'Mortgage Insurance Percentage (MI %)', 'MortgageInsuranceFlag',
       'Units_1', 'Units_2', 'Units_3', 'Units_4', 'OccupancyStatus_I',
       'OccupancyStatus_P', 'OccupancyStatus_S',
       'Original Combined Loan-to-Value (CLTV)',
       'Original Debt-to-Income (DTI) Ratio', 'Original UPB',
       'Original Loan-to-Value (LTV)', 'Original Interest Rate', 'Channel_B',
       'Channel_C', 'Channel_R', 'PropertyType_CO', 'PropertyType_CP',
       'PropertyType_MH', 'PropertyType_PU', 'PropertyType_SF',
       'LoanPurpose_C', 'LoanPurpose_N', 'LoanPurpose_P', 'LoanTerm_360',
       'LoanTerm_180', 'LoanTerm_240', 'LoanTerm_120', 'LoanTerm_300',
       'Original Loan Term', 'OneBorrower', 'AffordableProgramFlag'],
      dtype='object')

In [3]:
training_dat['Loan Deliquency Within Year'].value_counts()

0.0    387832
1.0      8435
Name: Loan Deliquency Within Year, dtype: int64

In [4]:
# tf = training_dat['Loan Deliquency Within Year'] == 1.0
# true_dat = training_dat[tf].copy()
# false_dat = training_dat[~tf].sample(len(true_dat)).copy()
# balanced_dat = pd.concat([true_dat,false_dat])

In [5]:
metrics = training_dat.describe().transpose()
metrics.reset_index(inplace=True)
metrics.rename(columns={'index':'column'},inplace=True)
metrics.head()

,column,count,mean,std,min,25%,50%,75%,max
0,Loan Deliquency Within Year,396267.0,0.021286,0.144337,0.0,0.000000,0.000000,0.000000,1.0
1,Credit Score,396267.0,0.721038,0.127547,0.0,0.634503,0.748538,0.824561,1.0
2,Mortgage Insurance Percentage (MI %),396267.0,0.138525,0.272966,0.0,0.000000,0.000000,0.000000,1.0
3,MortgageInsuranceFlag,396267.0,0.218746,0.413397,0.0,0.000000,0.000000,0.000000,1.0
4,Units_1,396267.0,0.980379,0.138693,0.0,1.000000,1.000000,1.000000,1.0


In [7]:
# tf1 = metrics['mean'] < 1.1
# tf2 = metrics['mean'] > -.1
# tf = tf1 & tf2
# metrics[tf]

In [7]:
# Only keep columns with diverse selection of values
keep_cols = ['Loan Sequence Number']
exclude_cols = ['PropertyType_CP']
good_cols = list(metrics[tf]['column'])
keep_cols.extend(good_cols)
keep_cols

['Loan Sequence Number',
 'Loan Deliquency Within Year',
 'Credit Score',
 'Mortgage Insurance Percentage (MI %)',
 'Units_1',
 'Units_2',
 'Units_3',
 'Units_4',
 'OccupancyStatus_I',
 'OccupancyStatus_P',
 'OccupancyStatus_S',
 'Original Combined Loan-to-Value (CLTV)',
 'Original Debt-to-Income (DTI) Ratio',
 'Original UPB',
 'Original Loan-to-Value (LTV)',
 'Original Interest Rate',
 'Channel_B',
 'Channel_C',
 'Channel_R',
 'PropertyType_CO',
 'PropertyType_CP',
 'PropertyType_MH',
 'PropertyType_PU',
 'PropertyType_SF',
 'LoanPurpose_C',
 'LoanPurpose_N',
 'LoanPurpose_P',
 'LoanTerm_360',
 'LoanTerm_180',
 'LoanTerm_240',
 'LoanTerm_120',
 'LoanTerm_300',
 'Original Loan Term',
 'OneBorrower',
 'AffordableProgramFlag']

In [8]:
final_df = training_dat[keep_cols].copy()
final_df.head()

,Loan Sequence Number,Loan Deliquency Within Year,Credit Score,Mortgage Insurance Percentage (MI %),Units_1,Units_2,Units_3,Units_4,OccupancyStatus_I,OccupancyStatus_P,...,LoanPurpose_N,LoanPurpose_P,LoanTerm_360,LoanTerm_180,LoanTerm_240,LoanTerm_120,LoanTerm_300,Original Loan Term,OneBorrower,AffordableProgramFlag
0,F110Q1000008,0.0,0.865497,0.0,1,0,0,0,0,1,...,1,0,1,0,0,0,0,0.592885,0,0
1,F110Q1000064,0.0,0.900585,0.0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0.592885,1,0
2,F110Q1000072,0.0,0.804094,0.0,1,0,0,0,0,1,...,1,0,1,0,0,0,0,0.592885,0,0
3,F110Q1000080,0.0,0.464912,0.0,1,0,0,0,0,1,...,1,0,1,0,0,0,0,0.592885,1,0
4,F110Q1000096,0.0,0.850877,0.0,1,0,0,0,0,1,...,1,0,1,0,0,0,0,0.592885,1,0


In [9]:
idx_col = 'Loan Sequence Number'

test_size = round(len(final_df[idx_col]) * .2)
test_loans = final_df[idx_col].sample(test_size)
train_loans = list(set(final_df[idx_col].unique()) - set(test_loans))

train_df = pd.DataFrame({idx_col:train_loans})
train_df = train_df.merge(final_df, on=idx_col, how='left')

test_df = pd.DataFrame({idx_col:test_loans})
test_df = test_df.merge(final_df, on=idx_col, how='left')

In [10]:
train_df.head()

,Loan Sequence Number,Loan Deliquency Within Year,Credit Score,Mortgage Insurance Percentage (MI %),Units_1,Units_2,Units_3,Units_4,OccupancyStatus_I,OccupancyStatus_P,...,LoanPurpose_N,LoanPurpose_P,LoanTerm_360,LoanTerm_180,LoanTerm_240,LoanTerm_120,LoanTerm_300,Original Loan Term,OneBorrower,AffordableProgramFlag
0,F118Q1227575,0.0,0.538012,0.0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0.592885,1,0
1,F112Q1025871,0.0,0.614035,0.0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0.592885,0,0
2,F117Q2099665,0.0,0.631579,0.0,1,0,0,0,0,1,...,0,1,1,0,0,0,0,0.592885,0,0
3,F113Q2015308,0.0,0.859649,0.0,1,0,0,0,1,0,...,0,1,1,0,0,0,0,0.592885,1,0
4,F118Q2103288,0.0,0.540936,0.0,1,0,0,0,0,1,...,1,0,0,1,0,0,0,0.237154,1,0


In [11]:
test_df.head()

,Loan Sequence Number,Loan Deliquency Within Year,Credit Score,Mortgage Insurance Percentage (MI %),Units_1,Units_2,Units_3,Units_4,OccupancyStatus_I,OccupancyStatus_P,...,LoanPurpose_N,LoanPurpose_P,LoanTerm_360,LoanTerm_180,LoanTerm_240,LoanTerm_120,LoanTerm_300,Original Loan Term,OneBorrower,AffordableProgramFlag
0,F114Q1047222,0.0,0.622807,0.0,1,0,0,0,0,1,...,0,1,1,0,0,0,0,0.592885,0,0
1,F110Q2176065,0.0,0.856725,0.0,1,0,0,0,0,1,...,0,1,1,0,0,0,0,0.592885,0,0
2,F113Q3312597,0.0,0.593567,0.0,1,0,0,0,0,1,...,1,0,1,0,0,0,0,0.592885,0,0
3,F119Q3422102,0.0,0.921053,0.0,1,0,0,0,0,1,...,1,0,1,0,0,0,0,0.592885,1,0
4,F113Q2272587,0.0,0.815789,0.0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0.237154,0,0


In [12]:
# Build training and test datasets
X_train = train_df[train_df.columns[2:]].to_numpy()
y_train = train_df["Loan Deliquency Within Year"].to_numpy()

X_test = test_df[test_df.columns[2:]].to_numpy()
y_test = test_df["Loan Deliquency Within Year"].to_numpy()

In [13]:
X_train.shape

(317014, 33)

In [14]:
# From https://aakashgoel12.medium.com/how-to-add-user-defined-function-get-f1-score-in-keras-metrics-3013f979ce0d#:~:text=By%20default%2C%20f1%20score%20is%20not%20part%20of,each%20epoch%2C%20need%20to%20make%20some%20more%20efforts.
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# From https://medium.com/dataseries/how-to-deal-with-unbalanced-dataset-in-binary-classification-part-3-e580d8d09883
def weighted_binary_crossentropy(y_true, y_pred):
    weights = (y_true * 9) + 1
    bce = K.binary_crossentropy(y_true, y_pred)
    weighted_bce = K.mean(bce * weights)
    return weighted_bce

In [18]:
# Build model
# Taken from https://www.kaggle.com/jashsheth5/binary-classification-with-sklearn-and-keras-95

model = Sequential()
model.add(Dense(33,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.BinaryAccuracy(name="binary_accuracy"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall")
]
model.compile(loss=weighted_binary_crossentropy,optimizer='adam',metrics=metrics)

In [19]:
history = model.fit(X_train,y_train,batch_size=2048,
                    epochs=30, validation_data=(X_test,y_test), verbose=1)

Train on 317014 samples, validate on 79253 samples
Epoch 1/30
317014/317014 [==============================] - 1s 3us/step - loss: 0.5845 - fn: 6590.0000 - fp: 6109.0000 - tn: 304178.0000 - tp: 137.0000 - binary_accuracy: 0.9599 - precision: 0.0219 - recall: 0.0204 - val_loss: 0.5500 - val_fn: 1708.0000 - val_fp: 0.0000e+00 - val_tn: 77545.0000 - val_tp: 0.0000e+00 - val_binary_accuracy: 0.9784 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/30
317014/317014 [==============================] - 1s 2us/step - loss: 0.5339 - fn: 6727.0000 - fp: 0.0000e+00 - tn: 310287.0000 - tp: 0.0000e+00 - binary_accuracy: 0.9788 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.5320 - val_fn: 1708.0000 - val_fp: 0.0000e+00 - val_tn: 77545.0000 - val_tp: 0.0000e+00 - val_binary_accuracy: 0.9784 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/30
317014/317014 [==============================] - 1s 2us/step - loss: 0.5202 - fn: 6726.0000 - fp: 3.0000 - tn: 310284.0000 - tp

In [ ]:
history.history.keys()

In [ ]:
train_accuracy = history.history['precision'][-1]
val_accuracy = history.history['val_precision'][-1]

print("Training accuracy: " + str(train_accuracy))
print("Validation accuracy: " + str(val_accuracy))

In [ ]:
preds = model.predict(X_test)
y_pred = np.where(preds>0.5,1,0)
confusion_matrix(y_test, y_pred)

In [ ]:
import lime.lime_tabular

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=final_df.columns[2:], class_names=["Delinquent","Not Delinquent"], discretize_continuous=False)

In [ ]:
test_df[test_df['OneBorrower']==0].head()

In [ ]:
len(X_test[3722])

In [ ]:
preds

In [ ]:
def predict_both(x):
    val = model.predict(x)
    
    new_vals = []
    for i in val:
        new_val = np.array([i[0],1-i[0]])
        new_vals.append(new_val)
    
    return np.array(new_vals)

In [ ]:
preds[3]

In [ ]:
predict_both(np.array([np.array([1]+[0 for _ in range(32)])]))

In [ ]:
exp.as_list()

In [ ]:
sum([val[1] for val in exp.as_list()])

In [ ]:
# np.array([0]+[0 for _ in range(32)])
exp = explainer.explain_instance(np.array([1]+[0 for _ in range(32)]), predict_both, num_samples=len(X_train), num_features=33, top_labels=1)
exp.show_in_notebook(show_all=True)